# DATASET PREPARATION

In [3]:
import tensorflow as tf
import numpy as np
import os
import shutil
import albumentations as A

In [4]:
imagePath = 'Dataset/png_images/IMAGES/'
maskPath = 'Dataset/png_masks/MASKS/'

valImagePath = 'ValDataset/png_images/IMAGES/'
valMaskPath = 'ValDataset/png_masks/MASKS/'

In [5]:
for image in os.listdir(maskPath):
    initPath = maskPath + image
    renamedPath = maskPath + 'img' + image[3:]
    os.rename(initPath, renamedPath)

## Splitting The Dataset

In [6]:
! mkdir ValDataset/
! mkdir ValDataset/png_images/
! mkdir ValDataset/png_masks/
! mkdir ValDataset/png_images/IMAGES/
! mkdir ValDataset/png_masks/MASKS/

In [7]:
valList=["0115","0025","0010","0003","0125","0200","0515","0225","0805","0915","0630","0301","0112","0905","0823","0527","0088","0055","0018",
          "0222","0049","0273","0299","0282","0372","0027","0445","0582","0374","0956","0211","0019","0961","0397","0699","0789","0996","0290",
          "0110","0315","0335","0419","0666","0525","0927","0555","0275","0855","0815","0130","0371","0412","0105","0423","0507","0028","0035","0118",
          "0232","0849","0673","0688","0777","0472","0991","0485","0592","0334","0827","0651","0619","0567","0393","0609","0719","0916","0190",
          ]

In [8]:
for name in valList:
    initPath = imagePath + 'img_' + name + '.png'
    finalPath = valImagePath + 'img_' + name + '.png'
    shutil.move(initPath, finalPath)
    
    initPath = maskPath + 'img_' + name + '.png'
    finalPath = valMaskPath + 'img_' + name + '.png'
    shutil.move(initPath, finalPath)

In [9]:
trainDataset = tf.data.Dataset.from_tensor_slices(
    ([imagePath+i for i in os.listdir(imagePath)],
    [maskPath+'img'+i[3:] for i in os.listdir(imagePath)])
)

valDataset = tf.data.Dataset.from_tensor_slices(
    ([valImagePath+i for i in os.listdir(valImagePath)],
    [valMaskPath+'img'+i[3:] for i in os.listdir(valImagePath)])
)

2025-02-08 23:22:56.555308: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-02-08 23:22:56.555479: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-02-08 23:22:56.555484: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
I0000 00:00:1739085776.555805 10627753 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1739085776.555940 10627753 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
for i in trainDataset.take(1):
    print(i)
for i in valDataset.take(1):
    print(i)

(<tf.Tensor: shape=(), dtype=string, numpy=b'Dataset/png_images/IMAGES/img_0564.png'>, <tf.Tensor: shape=(), dtype=string, numpy=b'Dataset/png_masks/MASKS/img_0564.png'>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'ValDataset/png_images/IMAGES/img_0996.png'>, <tf.Tensor: shape=(), dtype=string, numpy=b'ValDataset/png_masks/MASKS/img_0996.png'>)


2025-02-08 23:23:06.180332: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-02-08 23:23:06.183025: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [11]:
len(trainDataset), len(valDataset)

(923, 77)

## Normalizing The Dataset

In [12]:
#         R        G       B
MEAN = [123.675, 116.28, 103.53]
STD = [58.395, 57.12, 57.375]

# These Values Were Obtained From The ImageNet Dataset

In [13]:
def preProcess(imagePath, maskPath):
    image = tf.io.read_file(imagePath)
    image = tf.image.decode_jpeg(image)
    image = tf.cast(image, tf.float32)
    image = (image - MEAN) / STD

    mask = tf.io.read_file(maskPath)
    mask = tf.image.decode_jpeg(mask)
    mask = tf.squeeze(mask, axis=-1)
    mask = tf.cast(mask, tf.float32)
    
    return image, mask

In [14]:
prepTrainDataset = trainDataset.map(preProcess, num_parallel_calls=tf.data.AUTOTUNE)
prepValDataset = valDataset.map(preProcess, num_parallel_calls=tf.data.AUTOTUNE)

In [15]:
for i, j in prepTrainDataset.take(1):
    print(i.shape)
    print(j.shape)

(825, 550, 3)
(825, 550)


## Data Augmentation Using Albumentations

In [16]:
H,W = 512,512

In [17]:
transform = A.Compose([
    A.RandomCrop (H,W, p=1.0),
    A.HorizontalFlip(p=0.3),
    A.VerticalFlip(p=0.3),
    A.RandomRotate90(p=0.3),
    A.Transpose(p=0.3),
    A.Sharpen (alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.1),
    A.RandomShadow (shadow_roi=(0, 0.5, 1, 1),
                    num_shadows_lower=1, num_shadows_upper=2,
                    shadow_dimension=5, p=0.1),
    A.RandomBrightnessContrast(p=0.2),
    #A.Resize(H,W),
])

valTransform = A.Compose([
    A.Resize(H, W),
])

/var/folders/8v/j561wc497p1672n3xvshbbd00000gn/T/ipykernel_22730/1788790855.py:8: UserWarning: Argument(s) 'num_shadows_lower, num_shadows_upper' are not valid for transform RandomShadow
  A.RandomShadow (shadow_roi=(0, 0.5, 1, 1),


In [18]:
def augAlbument(image, mask):
    augmented = transform(image=image, mask=mask)
    
    return [tf.convert_to_tensor(augmented['image'], dtype=tf.float32), 
            tf.convert_to_tensor(augmented['mask'], dtype=tf.float32)]

def valAugAlbument(image, mask):
    augmented = valTransform(image=image, mask=mask)
    
    return [tf.convert_to_tensor(augmented['image'], dtype=tf.float32), 
            tf.convert_to_tensor(augmented['mask'], dtype=tf.float32)]

In [19]:
def augment(image, mask):
    augmentedOutput = tf.numpy_function(augAlbument, [image, mask], [tf.float32, tf.float32])
    return {'pixel_values': tf.transpose(augmentedOutput[0], (2, 0, 1)), 'labels': augmentedOutput[1]}

def valAugment(image, mask):
    augmentedOutput = tf.numpy_function(valAugAlbument, [image, mask], [tf.float32, tf.float32])
    return {'pixel_values': tf.transpose(augmentedOutput[0], (2, 0, 1)), 'labels': augmentedOutput[1]}

# Transpose Explained
# The Transpose Function Is Used To Rearrange The Dimensions Of The Image Tensor from (H,W,C) To (C,H,W)
# h, w, c      ->        c, h, w
# 0, 1, 2      ->        2, 0, 1

In [20]:
BATCH_SIZE = 2
trainDataset = (
    prepTrainDataset
    .shuffle(10)
    .map(augment,num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)
valDataset = (
    prepValDataset
    .map(valAugment,num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

In [21]:
for i in trainDataset.take(1):
    print(i['pixel_values'].shape)
    print(i['labels'].shape)

(2, 3, 512, 512)
(2, 512, 512)


2025-02-08 23:27:38.678980: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [22]:
trainDataset.save('SavedDataset/trainDataset')
valDataset.save('SavedDataset/valDataset')

In [23]:
def getFolderSize(folderPath):
    totalSize = 0
    for dirpath, dirnames, filenames in os.walk(folderPath):
        for filename in filenames:
            filePath = os.path.join(dirpath, filename)
            totalSize += os.path.getsize(filePath)
    return totalSize  # Size in bytes
trainSizeInBytes = getFolderSize("SavedDataset/trainDataset")
valSizeInBytes = getFolderSize("SavedDataset/valDataset")
print(f"Train Dataset Size: {trainSizeInBytes / (1024 * 1024):.2f} MB")
print(f"Validation Dataset Size: {valSizeInBytes / (1024 * 1024):.2f} MB")

Train Dataset Size: 3692.04 MB
Validation Dataset Size: 308.00 MB
